In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from util import *

from decouple import config
import requests
import time
import pickle


In [35]:
API_KEY=config("API_KEY")
init_headers={"Authorization":f"Token {API_KEY}"}
response = requests.get("https://lambda-treasure-hunt.herokuapp.com/api/adv/init/",headers=init_headers)
room_id = record_room_info(player_graph,response)

KeyError: None

In [7]:
infile = open("player_graph_1.pkl",'rb')
player_graph = pickle.load(infile)
infile.close()

In [8]:
len(player_graph)

230

In [36]:
room_id = response.json()["room_id"]
room_id

106

In [ ]:
while len(player_graph) < 500:
    
#     import pdb;
#     pdb.set_trace()
    
    print(room_id,len(player_graph))
    
    direction = None
    ud  = unexplored_directions(player_graph,room_id)
    
    
    cooldown = response.json()["cooldown"]
    time.sleep(cooldown)
    
    if len(response.json()["items"])>0:
        for ii in response.json()["items"]:
            if "treasure" in ii:
                pass
            else:
                response = pickup_treasure(ii,API_KEY)
                cooldown=response.json()["cooldown"]
                time.sleep(cooldown)
                break
                
            print(response.json()["messages"])
            time.sleep(response.json()["cooldown"])
    
    previous_room = room_id
    
    
    
    if len(response.json()["exits"])==1:
        direction = response.json()["exits"].pop()
        response = wise_wizard(direction,previous_room,API_KEY)
        
    elif len(ud)>=1:
        direction = ud.pop()
        response = movement(direction,API_KEY)
        
    elif len(ud) ==0:
        nr = find_unexplored_room(player_graph)
        path = bfs(room_id, nr, player_graph)
        path = path[1:]
        while len(path)>0:
            next_room = path.pop(0)
            direction = find_room_direction(response,player_graph, room_id, next_room)
            print(path)
            if len(path)>=0:
                previous_room = room_id
                response = wise_wizard(direction,next_room,API_KEY)
                time.sleep(response.json()["cooldown"])
                room_id = record_room_info(player_graph,response,direction=direction,previous_room=previous_room)
    pickle_graph(player_graph)

    
    room_id = record_room_info(player_graph,response,direction=direction,previous_room=previous_room)
    

106 264
[150, 166, 117, 108, 78, 22, 36]
[166, 117, 108, 78, 22, 36]


In [34]:
response.json()

{'cooldown': 11.231425999999999, 'errors': ['Cooldown Violation: +5s CD']}

### Sell Treasure

In [81]:
path = bfs(room_id, 1, player_graph)
path = path[1:]
while len(path)>0:
    next_room = path.pop(0)
    direction = find_room_direction(response,player_graph, room_id, next_room)
    if len(path)>=0:
        previous_room = room_id
        response = wise_wizard(direction,next_room,API_KEY)
        time.sleep(response.json()["cooldown"])
        room_id = record_room_info(player_graph,response,direction=direction,previous_room=previous_room)

In [15]:
inventory

[]

In [17]:
status = check_inventory(API_KEY)
status

{'name': 'User 20299',
 'cooldown': 1.0,
 'encumbrance': 0,
 'strength': 10,
 'speed': 10,
 'gold': 2000,
 'bodywear': None,
 'footwear': None,
 'inventory': [],
 'abilities': [],
 'status': [],
 'has_mined': False,
 'errors': [],
 'messages': []}

In [13]:
for ii in inventory:
    print(ii)
    if "treasure" in ii:
        response = sell_treasure(ii,API_KEY)    
        time.sleep(response.json()["cooldown"])

[]